Introduction

Sommaire

Scraping

In [ ]:
urls = ["https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2024-2025",
"https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2023-2024",
]

Nettoyage des données